### Setup

In [1]:
!pip install google-cloud-secret-manager
!pip install langchain
!pip install openai
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Load dataset

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
!gsutil cp gs://cnn-dailymail-data/validation-100.csv .

Copying gs://cnn-dailymail-data/validation-100.csv...
- [1 files][412.0 KiB/412.0 KiB]                                                
Operation completed over 1 objects/412.0 KiB.                                    


In [4]:
import pandas as pd
val_df = pd.read_csv("validation-100.csv")

### Run LangChain summarization pipeline

In [5]:
from google.cloud import secretmanager
client = secretmanager.SecretManagerServiceClient()
resource_name = "projects/708283936417/secrets/openai-api-key/versions/1"
response = client.access_secret_version(request={"name": resource_name})
OPENAI_API_KEY = response.payload.data.decode('UTF-8')

In [6]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [7]:
PROMPT_TEMPLATE = '''Here is a news article. Summarize this article into 3 or 4 bullet points. Use less than 75 words in total.

```{text}```

CONCISE SUMMARY:'''

PROMPT = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["text"])

In [8]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)

In [9]:
from tqdm import tqdm

predictions = {}

for _, row in tqdm(val_df.iterrows(), total=len(val_df)):
  doc = Document(page_content=row["article"])
  predictions[row["id"]] = chain.run([doc])

100%|██████████| 100/100 [11:35<00:00,  6.96s/it]


In [10]:
import pandas as pd

predictions_df = pd.DataFrame([
    {"id": k, "prediction": v}
    for k,v in predictions.items()
])

In [11]:
OUTPUT_FILE = "langchain-prompt-bullet-100.csv"
predictions_df.to_csv(OUTPUT_FILE, index=False)
!gsutil cp $OUTPUT_FILE gs://cnn-dailymail-predictions/

Copying file://langchain-prompt-bullet-100.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/42.8 KiB.                                     


### Compute ROUGE scores

In [12]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import numpy as np
from rouge_score import rouge_scorer
from tqdm import tqdm

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for _, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
  target = row.highlights
  prediction = predictions[row.id]
  score = scorer.score(target=target, prediction=prediction)
  rouge1_scores.append(score["rouge1"].fmeasure)
  rouge2_scores.append(score["rouge2"].fmeasure)
  rougeL_scores.append(score["rougeL"].fmeasure)

print(f'''
rouge 1: {np.average(rouge1_scores)}
rouge 2: {np.average(rouge2_scores)}
rouge L: {np.average(rougeL_scores)}
''')

100%|██████████| 100/100 [00:00<00:00, 439.37it/s]


rouge 1: 0.36996057187135606
rouge 2: 0.1376650094636557
rouge L: 0.23946291284011637

